In [1]:
# 笔记本目的:
# 笔记本的目的是查看数据集中提供的不同的预先训练的嵌入，并了解它们在模型构建过程中是如何有用的。
# 首先，让我们导入必要的模块并读取输入数据。

In [1]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
print('Train shape : ', train_df.shape)
print('Test shape : ',  test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [3]:
# 下一步工作如下:
# 将训练数据集分为训练样本和val样本。交叉验证是一个耗时的过程，所以让我们来做简单的val分割。
# 用“na”填充文本列中缺少的值
# 标记文本列并将其转换为向量序列
# 根据需要填充序列——如果文本中的单词数量大于'max_len'，则将其截断为'max_len'，或者如果文本中的单词数量小于'max_len'，则为其余的值添加0。

In [3]:
## split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

## some config values
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df['question_text'].fillna('_na_').values
val_X = val_df['question_text'].fillna('_na_').values
test_X = test_df['question_text'].fillna('_na_').values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [4]:
# 没有Pretrained Embeddings:
# 现在我们已经完成了所有必要的预处理步骤，我们可以首先训练一个双向GRU模型。
# 我们不会在这个模型中使用任何预先训练好的单词嵌入，这些嵌入将会从头开始学习。
# 请查看模型摘要以了解所使用的层的详细信息。

In [6]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0      

In [7]:
# 使用train samples对模型进行训练，并在验证集上监控。
# 这只是一个运行了两个时代的样本模型。更改epoch、batch_size和模型参数可能会得到更好的模型。

In [8]:
## Train the model
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2


 103424/1175509 [=>............................] - ETA: 14:09:15 - loss: 0.6929 - acc: 0.519 - ETA: 7:06:43 - loss: 0.6787 - acc: 0.734 - ETA: 4:45:48 - loss: 0.6657 - acc: 0.80 - ETA: 3:35:22 - loss: 0.6527 - acc: 0.83 - ETA: 2:53:06 - loss: 0.6397 - acc: 0.85 - ETA: 2:24:55 - loss: 0.6264 - acc: 0.87 - ETA: 2:04:47 - loss: 0.6125 - acc: 0.88 - ETA: 1:49:41 - loss: 0.5982 - acc: 0.88 - ETA: 1:37:56 - loss: 0.5831 - acc: 0.89 - ETA: 1:28:33 - loss: 0.5676 - acc: 0.90 - ETA: 1:20:52 - loss: 0.5521 - acc: 0.90 - ETA: 1:14:28 - loss: 0.5377 - acc: 0.90 - ETA: 1:09:03 - loss: 0.5225 - acc: 0.90 - ETA: 1:04:24 - loss: 0.5098 - acc: 0.91 - ETA: 1:00:23 - loss: 0.4959 - acc: 0.91 - ETA: 56:52 - loss: 0.4824 - acc: 0.9126 - ETA: 53:45 - loss: 0.4699 - acc: 0.91 - ETA: 51:00 - loss: 0.4579 - acc: 0.91 - ETA: 48:31 - loss: 0.4464 - acc: 0.91 - ETA: 46:18 - loss: 0.4355 - acc: 0.91 - ETA: 44:17 - loss: 0.4260 - acc: 0.91 - ETA: 42:27 - loss: 0.4190 - acc: 0.91 - ETA: 40:46 - loss: 0.4101 - acc: 0

 416768/1175509 [=========>....................] - ETA: 4:15 - loss: 0.1472 - acc: 0.948 - ETA: 4:15 - loss: 0.1471 - acc: 0.948 - ETA: 4:15 - loss: 0.1471 - acc: 0.948 - ETA: 4:14 - loss: 0.1471 - acc: 0.948 - ETA: 4:14 - loss: 0.1471 - acc: 0.948 - ETA: 4:14 - loss: 0.1470 - acc: 0.948 - ETA: 4:14 - loss: 0.1470 - acc: 0.948 - ETA: 4:13 - loss: 0.1469 - acc: 0.948 - ETA: 4:13 - loss: 0.1469 - acc: 0.948 - ETA: 4:13 - loss: 0.1468 - acc: 0.948 - ETA: 4:13 - loss: 0.1468 - acc: 0.948 - ETA: 4:12 - loss: 0.1468 - acc: 0.948 - ETA: 4:12 - loss: 0.1468 - acc: 0.948 - ETA: 4:12 - loss: 0.1467 - acc: 0.948 - ETA: 4:12 - loss: 0.1466 - acc: 0.948 - ETA: 4:11 - loss: 0.1466 - acc: 0.948 - ETA: 4:11 - loss: 0.1466 - acc: 0.948 - ETA: 4:11 - loss: 0.1466 - acc: 0.948 - ETA: 4:11 - loss: 0.1465 - acc: 0.948 - ETA: 4:11 - loss: 0.1463 - acc: 0.948 - ETA: 4:10 - loss: 0.1463 - acc: 0.948 - ETA: 4:10 - loss: 0.1462 - acc: 0.948 - ETA: 4:10 - loss: 0.1461 - acc: 0.948 - ETA: 4:10 - loss: 0.1461 - ac

 521216/1175509 [============>.................] - ETA: 3:31 - loss: 0.1396 - acc: 0.949 - ETA: 3:31 - loss: 0.1396 - acc: 0.949 - ETA: 3:31 - loss: 0.1396 - acc: 0.949 - ETA: 3:31 - loss: 0.1396 - acc: 0.949 - ETA: 3:30 - loss: 0.1395 - acc: 0.949 - ETA: 3:30 - loss: 0.1396 - acc: 0.949 - ETA: 3:30 - loss: 0.1395 - acc: 0.949 - ETA: 3:30 - loss: 0.1396 - acc: 0.949 - ETA: 3:30 - loss: 0.1395 - acc: 0.949 - ETA: 3:29 - loss: 0.1395 - acc: 0.949 - ETA: 3:29 - loss: 0.1394 - acc: 0.949 - ETA: 3:29 - loss: 0.1394 - acc: 0.949 - ETA: 3:29 - loss: 0.1393 - acc: 0.949 - ETA: 3:29 - loss: 0.1394 - acc: 0.949 - ETA: 3:28 - loss: 0.1393 - acc: 0.949 - ETA: 3:28 - loss: 0.1393 - acc: 0.949 - ETA: 3:28 - loss: 0.1392 - acc: 0.949 - ETA: 3:28 - loss: 0.1393 - acc: 0.949 - ETA: 3:28 - loss: 0.1392 - acc: 0.949 - ETA: 3:28 - loss: 0.1391 - acc: 0.949 - ETA: 3:27 - loss: 0.1391 - acc: 0.950 - ETA: 3:27 - loss: 0.1391 - acc: 0.950 - ETA: 3:27 - loss: 0.1391 - acc: 0.950 - ETA: 3:27 - loss: 0.1390 - ac

 625664/1175509 [==============>...............] - ETA: 2:55 - loss: 0.1352 - acc: 0.950 - ETA: 2:55 - loss: 0.1352 - acc: 0.950 - ETA: 2:55 - loss: 0.1352 - acc: 0.950 - ETA: 2:55 - loss: 0.1351 - acc: 0.950 - ETA: 2:55 - loss: 0.1351 - acc: 0.950 - ETA: 2:54 - loss: 0.1350 - acc: 0.950 - ETA: 2:54 - loss: 0.1350 - acc: 0.950 - ETA: 2:54 - loss: 0.1350 - acc: 0.950 - ETA: 2:54 - loss: 0.1350 - acc: 0.950 - ETA: 2:54 - loss: 0.1350 - acc: 0.950 - ETA: 2:54 - loss: 0.1349 - acc: 0.950 - ETA: 2:54 - loss: 0.1349 - acc: 0.950 - ETA: 2:53 - loss: 0.1349 - acc: 0.950 - ETA: 2:53 - loss: 0.1349 - acc: 0.950 - ETA: 2:53 - loss: 0.1349 - acc: 0.950 - ETA: 2:53 - loss: 0.1348 - acc: 0.950 - ETA: 2:53 - loss: 0.1348 - acc: 0.950 - ETA: 2:53 - loss: 0.1348 - acc: 0.950 - ETA: 2:52 - loss: 0.1348 - acc: 0.950 - ETA: 2:52 - loss: 0.1348 - acc: 0.950 - ETA: 2:52 - loss: 0.1347 - acc: 0.950 - ETA: 2:52 - loss: 0.1347 - acc: 0.950 - ETA: 2:52 - loss: 0.1347 - acc: 0.950 - ETA: 2:52 - loss: 0.1347 - ac

...

 835584/1175509 [====================>.........] - ETA: 1:41 - loss: 0.0977 - acc: 0.960 - ETA: 1:41 - loss: 0.0977 - acc: 0.960 - ETA: 1:41 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:40 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:39 - loss: 0.0977 - acc: 0.960 - ETA: 1:38 - loss: 0.0977 - acc: 0.960 - ETA: 1:38 - loss: 0.0977 - acc: 0.960 - ETA: 1:38 - loss: 0.0977 - ac

 941568/1175509 [=======================>......] - ETA: 1:17 - loss: 0.0975 - acc: 0.960 - ETA: 1:17 - loss: 0.0975 - acc: 0.960 - ETA: 1:17 - loss: 0.0975 - acc: 0.960 - ETA: 1:17 - loss: 0.0975 - acc: 0.960 - ETA: 1:17 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:16 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:15 - loss: 0.0975 - acc: 0.960 - ETA: 1:14 - loss: 0.0975 - acc: 0.961 - ETA: 1:14 - loss: 0.0975 - ac

1051648/1175509 [=========================>....] - ETA: 53s - loss: 0.0977 - acc: 0.96 - ETA: 53s - loss: 0.0977 - acc: 0.96 - ETA: 53s - loss: 0.0977 - acc: 0.96 - ETA: 52s - loss: 0.0977 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 52s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 51s - loss: 0.0978 - acc: 0.96 - ETA: 50s - loss: 0.0978 - acc: 0.96 - ETA: 50s - loss: 0.0978 - acc: 0.96 - ETA: 50s - loss: 0.0978 - acc: 0.96 - ETA: 50s - loss: 0.0978 - acc: 0.96 - ETA: 50s - loss: 0.0978 - acc: 0.96 -

1161728/1175509 [============================>.] - ETA: 28s - loss: 0.0977 - acc: 0.96 - ETA: 28s - loss: 0.0977 - acc: 0.96 - ETA: 27s - loss: 0.0977 - acc: 0.96 - ETA: 27s - loss: 0.0977 - acc: 0.96 - ETA: 27s - loss: 0.0978 - acc: 0.96 - ETA: 27s - loss: 0.0978 - acc: 0.96 - ETA: 27s - loss: 0.0977 - acc: 0.96 - ETA: 27s - loss: 0.0977 - acc: 0.96 - ETA: 27s - loss: 0.0978 - acc: 0.96 - ETA: 27s - loss: 0.0978 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 26s - loss: 0.0977 - acc: 0.96 - ETA: 25s - loss: 0.0977 - acc: 0.96 - ETA: 25s - loss: 0.0977 - acc: 0.96 - ETA: 25s - loss: 0.0978 - acc: 0.96 - ETA: 25s - loss: 0.0977 - acc: 0.96 - ETA: 25s - loss: 0.0978 - acc: 0.96 - ETA: 25s - loss: 0.0978 - acc: 0.96 -

1175509/1175509 [==============================] - ETA: 3s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 2s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 1s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 - ETA: 0s - loss: 0.0979 - acc: 0.960 -

In [9]:
#现在让我们得到验证样本预测和F1得分的最佳阈值。

In [16]:
pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int))))

130613/130613 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  -

In [18]:
#现在让我们也得到测试集预测并保存它们

In [20]:
pred_noemb_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA: 18 - ETA: 10 - ETA: 8 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 67us/step


In [21]:
#现在我们的模型构建已经完成，在进行下一步之前清理一些内存可能是一个好主意。

In [22]:
del model, inp, x
import gc
gc.collect()
time.sleep(10)

In [23]:
#所以我们得到了一些没有经过预先训练的嵌入的基线GRU模型。
#现在让我们使用所提供的嵌入并重新构建模型来查看性能

In [25]:
import os
print(os.listdir('../input/embeddings'))

['glove.840B.300d', 'GoogleNews-vectors-negative300', 'paragram_300_sl999', 'ReadMe.txt', 'wiki-news-300d-1M']


We have four different types of embeddings.
 * GoogleNews-vectors-negative300 - https://code.google.com/archive/p/word2vec/
 * glove.840B.300d - https://nlp.stanford.edu/projects/glove/
 * paragram_300_sl999 - https://cogcomp.org/page/resource_view/106
 * wiki-news-300d-1M - https://fasttext.cc/docs/en/english-vectors.html
 
 A very good explanation for different types of embeddings are given in this [kernel](https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge). Please refer the same for more details..

**Glove Embeddings:**

In this section, let us use the Glove embeddings and rebuild the GRU model.

In [9]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='utf-8') if len(o)>100 )

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_______________________________________________________

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

In [ ]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print('F1 score at threshold {0} is {1}'.format(thresh, metrics.fi_score(val_y, (pred_glove_val_y>thresh).astype(int))))

结果似乎比没有预训练Embedding的模型要好。

In [ ]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [10]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc
gc.collect()
time.sleep(10)

Wiki News FastText Embeddings:
现在，让我们使用在Wiki新闻语料库上训练的快速文本嵌入来代替手套嵌入，并重新构建模型。

In [ ]:
EMBDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='utf-8') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

inp = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metircs=['accuracy'])

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

In [ ]:
pred_fasttext_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print('F1 score at threshold {0} is {1}'.format(thresh, metrics.f1_score(val_y, (pred_fasttext_val_y>thresh).astype(int))))

In [ ]:
pred_fasttext_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
del word_index, embeddings_index, all_embs, embedding_matirx, model, inp, x
import gc
fc.collect()
time.sleep(10)

Paragram嵌入的:
    
在本节中，我们可以使用parram嵌入来构建模型并进行预测。

In [ ]:
EMBEDDING_FILE = '../input/embedings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='utf-8', errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embsshape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matirx = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

inp = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = BIdirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

In [ ]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)#四舍五入到小数第二位
    print('F1 score at threshold {0} is {1}'.format(thresh, metrics.fi_score(val_y, (pred_paragram_val_y>thresh).astype(int))))

In [ ]:
pred_paragram_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc
gc.collect()
time.sleep(10)

观察:<br>
整体的预训练Embedding似乎比非预训练模型得到更好的结果。<br>
不同的预训练Embedding的性能几乎是相似的。<br>
最后的融合:<br>
尽管具有不同预训练嵌入的模型的结果是相似的，但是它们很可能从数据中捕获不同类型的信息。让我们把这三种模型的预测平均起来。

In [ ]:
pred_val_y = 0.33*pred_glove_val_y + 0.33*pred_fasttext_val_y + 0.34*pred_paragram_val_y
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print('F1 score at threshold {0} is {1}'.format(thresh, metrics.f1_score(val_y, (pred_cal_y>thresh).astype(int))))

结果似乎比单独的预训练模型要好，因此我们让我们使用这个模型混合创建一个提交文件。

In [ ]:
pred_test_y = 0.33*pred_glove_test_y + 0.33*pred_fasttext_test_y + 0.34*pred_paragram_test_y
pred_test_y = (pred_test_y>0.35).astype(int)
out_df = pd.Dataframe({'qid':test_df['qid'].values})
out_df['prediction'] = pred_test_y
out_df.to_csv('submission.csv', index=False)


**References:**

Thanks to the below kernels which helped me with this one. 
1. https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout
2. https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge